In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import seaborn as sb
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from numba import vectorize
import unidecode
from nltk.corpus import stopwords
import time
import spacy
import re

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.metrics import classification_report



import string,lxml,bs4,nltk
from warnings import simplefilter

In [2]:
contractions = {
"ain't": "are not",
"aren't": "am not",
"can't": "can not",
"can't've": "can not have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I had",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [3]:
def load_and_prapare(fileName):
    
    # READ FILE TO DATAFRAME
    result = pd.read_csv(fileName)
    
    # Remove un necessary columns
    columns_needed = ['ProductId','Score','Summary','Text']
    to_drop_columns = list(x for x in result.columns.tolist() if x not in columns_needed)
    result.drop(to_drop_columns,axis=1,inplace=True)
    
    # Remove Nulls
    columns = ['Summary']
    for column in columns:
        result[column].fillna('', inplace=True)
     
    # Merge Summary and Text
    result['review'] = result['Summary'] + ' ' + result['Text'] 
    result.drop(['Summary','Text','ProductId'],axis=1,inplace=True)
    
    # Remove Outliers
    result['length'] = result['review'].apply(len)
    result[result['length'] < 3000]
    result.drop(['length'],axis=1,inplace=True)
    
    result.drop_duplicates(keep="first",inplace=True)
    
    return result


In [150]:
def text_processing(text):
    
    # Transform to Lower characters
    text = text.lower()
    
    # Remove html tags
    sp = BeautifulSoup(text, "html.parser")
    text = sp.get_text(separator=" ")
    
    # Remove accent
    text = unidecode.unidecode(text)
    
    # Remove pucntuations
    for punc in string.punctuation.replace("'",""):
        if punc in text:
            text = text.replace(punc," ")
    
    # Remove Spaces
    text = " ".join(text.split())
    
    # Expand short words
    for word in text.split():
        if word.lower() in contractions:
            text = text.replace(word, contractions[word.lower()])
    
    # Replace '
    text = text.replace("'", " ")
    
    # Remove Spaces
    text = " ".join(text.split())
    
    # Remove stopwords
    sWords = set(stopwords.words('english'))
    sWords = list(sWords)
    deselect_stop_words = ['no', 'not']

    for item in deselect_stop_words:
        sWords.remove(item)
    for word in sWords:
        text = text.replace(" " + word.lower() + " "," ")
        
    # Remove Spaces
    text = " ".join(text.split())
    
    text = re.sub("\S*\d\S*", "", text).strip()
         
    # Lemmatize words spacy       
    #nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    #doc = nlp(text)
    #lemmed = [token.lemma_ for token in doc]
    #lemmed = ' '.join(lemmed) 
    
    # Lemmatize words wordnet
    #lemmatizer = WordNetLemmatizer()
    #lemmed = [lemmatizer.lemmatize(word) for word in text.split()]
    #lemmed = ' '.join(lemmed) 
    
    
    stemmer = SnowballStemmer("english")
    stemmed = [stemmer.stem(word) for word in text.split()]
    stemmed = ' '.join(stemmed) 
       
            
    # Tokenize
    tokenized = word_tokenize(stemmed)
    
    return tokenized
    #return text
    

In [337]:
original_df = load_and_prapare('train.csv')

In [340]:
original_df_sub = original_df.sample(frac=1).head(10000).copy()

In [341]:
pipline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_processing,max_features=2000,ngram_range=(1,2))),
    ('tfidf',TfidfTransformer()),
    #('classifier',MultinomialNB())
    ('classifier',LogisticRegression())
    #('classifier',SGDClassifier())
])

In [342]:
x_train, x_test, y_train, y_test = train_test_split(original_df_sub['review'],original_df_sub['Score'],test_size=0.3)

In [343]:
pipline.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function text_processing at 0x00000258BBF6FD90>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=2000, min_df=1, ngram_range=(1, 2), preproce...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [344]:
predictions = pipline.predict(x_test)

In [345]:
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix
print ("Accuracy:", accuracy_score(y_test, predictions))
#print ("Precision:", precision_score(y_test, predictions))
print (classification_report(y_test, predictions))
print (confusion_matrix(y_test, predictions))

Accuracy: 0.6896666666666667
              precision    recall  f1-score   support

           1       0.61      0.43      0.50       287
           2       0.56      0.03      0.06       169
           3       0.37      0.08      0.14       201
           4       0.43      0.11      0.18       441
           5       0.71      0.99      0.83      1902

   micro avg       0.69      0.69      0.69      3000
   macro avg       0.53      0.33      0.34      3000
weighted avg       0.63      0.69      0.61      3000

[[ 123    0   10    5  149]
 [  37    5   12   16   99]
 [  21    3   17   26  134]
 [  15    0    7   50  369]
 [   7    1    0   20 1874]]


### with Grid Search

In [346]:
original_df_sub_2 = original_df.sample(frac=1).head(1000).copy()

In [347]:
x_train, x_test, y_train, y_test = train_test_split(original_df_sub_2['review'],original_df_sub_2['Score'],test_size=0.3)

In [348]:
estimators = [LogisticRegression,MultinomialNB,SGDClassifier]

In [349]:
def create_pipeline(estimator):
    
    pipeline_steps = [
        
        ('cv',CountVectorizer()),
        ('tfidf',TfidfTransformer()),
        ('classifier',estimator)
    ]
    
    return Pipeline(pipeline_steps)

In [350]:
#  parameters for Grid Search (CV and TfIdf) - BASE 
param_grid= {}

#Parameters for CountVectorizer
param_grid.update({'cv__ngram_range':[(1,1),(2,2)]})
#param_grid.update({'cv__stop_words':[None,'english']})
#param_grid.update({'cv__max_df':[1,2,3,4]})
param_grid.update({'cv__analyzer':[text_processing]})
param_grid.update({'cv__max_features':[2000]})


#Parameters of TFIDF
param_grid.update({'tfidf__use_idf':[True,False]})

# Parameters for LR
param_grid_LR = {}
param_grid_LR = {'classifier__C':[0.0001]}
param_grid_LR.update(param_grid)

# Parameters for MN
param_grid_MN = {}
param_grid_MN = {'classifier__alpha':[0.0001,0.001]}
param_grid_MN.update(param_grid)

#Parameters for SGD here
param_grid_SG={}
param_grid_SG.update(param_grid)



In [351]:
estimators = {
    LogisticRegression : param_grid_LR,
    MultinomialNB : param_grid_MN,
    SGDClassifier : param_grid_SG
}

In [352]:
def classification_report_final (model): 
    y_pred_final = model.predict(df_test_data['review'])
    score_final =accuracy_score(df_test_labels,y_pred_final)
    print('Model score on unseen data',score_final)
    print (classification_report(df_test_labels,y_pred_final))

In [353]:
df_test_data = load_and_prapare("test.csv").head(1000)
df_test_labels = pd.read_csv("labels.csv",usecols=['Score']).head(1000)

In [354]:
df_test_data.head(10)

,review
0,Love this flavor Fog chaser was the best both ...
1,"Tasty Tea We found this at PF Changs, and it t..."
2,Yum Not quite a chocolate bar substitute but d...
3,Gluten Free Biscotti This is not as tasty as P...
4,"Happy Puppies <a href=""http://www.amazon.com/g..."
5,NASTY TASTING!!!!!!!!!!!!!!!!!!!!!!!!!!! THIS ...
6,This THE BEST formula to use ..by FAR - This i...
7,My Schnauzer Loves This Food! I have a miniatu...
8,Less Tasty Than Expected. Like other reviewers...
9,Best gummies EVER! I've looked high and low fo...


In [355]:
for estimator,search_param in estimators.items():
    scores=[]
    model = create_pipeline(estimator())
    search=GridSearchCV(model,search_param,n_jobs=-1)
    search.fit(x_train,y_train)
    y_pred = search.predict(x_test)
    score=accuracy_score(y_test,y_pred)
    scores.append(score)
    print(estimator.__name__,'scored',score)
    print("Best parameter (CV score=%0.3f):" % search.best_score_)
    print(search.best_params_)
    print (classification_report(y_test,y_pred))
    print('\n')
    print('Cross validation with unseen test data')
    classification_report_final(search)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression scored 0.6333333333333333
Best parameter (CV score=0.649):
{'classifier__C': 0.0001, 'cv__analyzer': <function text_processing at 0x00000258BBF6FD90>, 'cv__max_features': 2000, 'cv__ngram_range': (1, 1), 'tfidf__use_idf': True}
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        26
           2       0.00      0.00      0.00        14
           3       0.00      0.00      0.00        20
           4       0.00      0.00      0.00        50
           5       0.63      1.00      0.78       190

   micro avg       0.63      0.63      0.63       300
   macro avg       0.13      0.20      0.16       300
weighted avg       0.40      0.63      0.49       300



Cross validation with unseen test data


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model score on unseen data 0.682
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        80
           2       0.00      0.00      0.00        46
           3       0.00      0.00      0.00        71
           4       0.00      0.00      0.00       121
           5       0.68      1.00      0.81       682

   micro avg       0.68      0.68      0.68      1000
   macro avg       0.14      0.20      0.16      1000
weighted avg       0.47      0.68      0.55      1000



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(

MultinomialNB scored 0.64
Best parameter (CV score=0.654):
{'classifier__alpha': 0.001, 'cv__analyzer': <function text_processing at 0x00000258BBF6FD90>, 'cv__max_features': 2000, 'cv__ngram_range': (1, 1), 'tfidf__use_idf': False}
              precision    recall  f1-score   support

           1       1.00      0.08      0.14        26
           2       0.00      0.00      0.00        14
           3       0.00      0.00      0.00        20
           4       0.40      0.04      0.07        50
           5       0.65      0.99      0.78       190

   micro avg       0.64      0.64      0.64       300
   macro avg       0.41      0.22      0.20       300
weighted avg       0.56      0.64      0.52       300



Cross validation with unseen test data
Model score on unseen data 0.671
              precision    recall  f1-score   support

           1       0.22      0.03      0.04        80
           2       0.00      0.00      0.00        46
           3       0.00      0.00      0.0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier scored 0.6466666666666666
Best parameter (CV score=0.643):
{'cv__analyzer': <function text_processing at 0x00000258BBF6FD90>, 'cv__max_features': 2000, 'cv__ngram_range': (2, 2), 'tfidf__use_idf': True}
              precision    recall  f1-score   support

           1       0.56      0.38      0.45        26
           2       0.00      0.00      0.00        14
           3       0.00      0.00      0.00        20
           4       0.41      0.34      0.37        50
           5       0.74      0.88      0.80       190

   micro avg       0.65      0.65      0.65       300
   macro avg       0.34      0.32      0.33       300
weighted avg       0.59      0.65      0.61       300



Cross validation with unseen test data
Model score on unseen data 0.542
              precision    recall  f1-score   support

           1       0.17      0.10      0.13        80
           2       0.02      0.02      0.02        46
           3       0.14      0.07      0.09        71
   